In [1]:
import numpy as np
import scipy.io

In [5]:
mat_dict = scipy.io.loadmat('data_structure_ANM210861_20130701.mat')

In [7]:
# for key in mat_dict:
#     print(key)
    
data = mat_dict['obj']
# print(mat_dict)

data = data['trialTypeMat'][0,0]

# d = data['waveforms']
# val = d[0,0]
# print(val)
# print(d[0,0][:,:50])


valid_trials = []
labels = []
for i in range(len(data[0,:])):
    if data[1,i] == 1:
        labels.append(1)
        valid_trials.append(i+1)
    elif data[0,i] == 1:
        labels.append(0)
        valid_trials.append(i+1)
        
print(valid_trials)
print(labels)
# 1 means right, 0 menas left


file = open('mice_labels.npy', 'wb')
np.save(file, labels)
file.close()


[11, 14, 15, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 54, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 68, 69, 70, 71, 72, 73, 74, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 95, 96, 97, 98, 99, 100, 101, 102, 103, 105, 106, 107, 108, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 126, 127, 129, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 146, 147, 148, 149, 150, 151, 153, 157, 159, 160, 161, 163, 164, 165, 166, 168, 169, 170, 171, 172, 173, 175, 176, 177, 178, 179, 180, 182, 183, 184, 185, 186, 187, 188, 189, 190, 192, 193, 194, 195, 198, 199, 201, 204, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 245, 246, 247, 248, 249, 250, 252, 253, 254, 256, 257, 258, 259, 260, 261, 263, 264,

In [10]:
mat_dict = scipy.io.loadmat('data_structure_ANM210861_20130701.mat')

In [4]:
# raw_trace_127_trial_5.mat

epochs = []
epochs = np.asarray(epochs)
min_timepoint = 1000000

for t in valid_trials:
    print(t)
    trial = scipy.io.loadmat('C:\\voltage_traces_ANM210861_20130701\\raw_trace_127_trial_' + str(t) + '.mat')['ch_MUA']
    timepoint, chan = trial.shape
    if min_timepoint > timepoint:
        min_timepoint = timepoint
    
    trial = trial[:min_timepoint, :]
    trial = trial.reshape((1, min_timepoint, chan))
    print(trial.shape)
    if epochs.size == 0:
        epochs = trial
    else:
        epochs = epochs[:, :min_timepoint, :]
        epochs = np.concatenate((epochs, trial), axis = 0)
    print(epochs.shape)




11
(1, 105546, 32)
(1, 105546, 32)
14
(1, 105546, 32)
(2, 105546, 32)
15
(1, 103772, 32)
(3, 103772, 32)
16
(1, 103752, 32)
(4, 103752, 32)
17
(1, 103752, 32)
(5, 103752, 32)
18
(1, 103752, 32)
(6, 103752, 32)
19
(1, 103752, 32)
(7, 103752, 32)
20
(1, 103752, 32)
(8, 103752, 32)
23
(1, 103752, 32)
(9, 103752, 32)
24
(1, 103752, 32)
(10, 103752, 32)
25
(1, 103343, 32)
(11, 103343, 32)
26
(1, 103343, 32)
(12, 103343, 32)
27
(1, 103343, 32)
(13, 103343, 32)
28
(1, 103343, 32)
(14, 103343, 32)
29
(1, 103343, 32)
(15, 103343, 32)
30
(1, 103343, 32)
(16, 103343, 32)
31
(1, 103343, 32)
(17, 103343, 32)
32
(1, 103343, 32)
(18, 103343, 32)
33
(1, 103343, 32)
(19, 103343, 32)
34
(1, 103343, 32)
(20, 103343, 32)
35
(1, 103343, 32)
(21, 103343, 32)
36
(1, 103343, 32)
(22, 103343, 32)
37
(1, 103343, 32)
(23, 103343, 32)
38
(1, 103343, 32)
(24, 103343, 32)
39
(1, 103343, 32)
(25, 103343, 32)
40
(1, 103343, 32)
(26, 103343, 32)
41
(1, 103343, 32)
(27, 103343, 32)
42
(1, 103343, 32)
(28, 103343, 32)
4

(222, 103342, 32)
272
(1, 103342, 32)
(223, 103342, 32)
273
(1, 103342, 32)
(224, 103342, 32)
274
(1, 103342, 32)
(225, 103342, 32)
275
(1, 103342, 32)
(226, 103342, 32)
276
(1, 103342, 32)
(227, 103342, 32)
277
(1, 103342, 32)
(228, 103342, 32)
278
(1, 103342, 32)
(229, 103342, 32)
279
(1, 103342, 32)
(230, 103342, 32)
280
(1, 103342, 32)
(231, 103342, 32)
281
(1, 103342, 32)
(232, 103342, 32)
282
(1, 103342, 32)
(233, 103342, 32)
283
(1, 103342, 32)
(234, 103342, 32)
284
(1, 103342, 32)
(235, 103342, 32)
285
(1, 103342, 32)
(236, 103342, 32)
286
(1, 103342, 32)
(237, 103342, 32)
287
(1, 103342, 32)
(238, 103342, 32)
288
(1, 103342, 32)
(239, 103342, 32)
289
(1, 103342, 32)
(240, 103342, 32)
290
(1, 103342, 32)
(241, 103342, 32)
291
(1, 103342, 32)
(242, 103342, 32)
292
(1, 103342, 32)
(243, 103342, 32)
293
(1, 103342, 32)
(244, 103342, 32)
294
(1, 103342, 32)
(245, 103342, 32)
296
(1, 103342, 32)
(246, 103342, 32)
297
(1, 103342, 32)
(247, 103342, 32)
299
(1, 103342, 32)
(248, 103342

In [24]:
trial, timepoint, channel = epochs.shape

In [11]:
file = open('mice_epochs.npy', 'wb')
np.save(file, epochs)
file.close()

In [22]:
epochs = np.load('mice_epochs.npy')



In [28]:
sparse_ind = [i*100 for i in range(timepoint//100) ]
print(len(sparse_ind))
sparse_epochs = epochs[:, sparse_ind, :]
sparse_epochs = np.swapaxes(sparse_epochs,1,2)

1033


In [2]:
sparse_epochs = np.load('sparse_mice_epochs.npy')
sparse_epochs = np.swapaxes(sparse_epochs,1,2)
labels = np.load('mice_labels.npy')

In [29]:
import sklearn 
import pyriemann.estimation
import pyriemann.classification
import sklearn.linear_model
from sklearn.model_selection import ShuffleSplit  
from scipy.fftpack import fft, ifft

cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
scores = []
epochs_data = sparse_epochs

for train_idx, test_idx in cv.split(epochs_data):
    y_train, y_test = np.asarray(labels)[train_idx], np.asarray(labels)[test_idx]
    X_train = epochs_data[train_idx]
    X_test = epochs_data[test_idx]
    print(X_train.shape)

    
    cov =  pyriemann.estimation.Covariances('lwf')
    cov_X_train = cov.transform(X_train)
    cov_X_test = cov.transform(X_test)
    
    TSclassifier = pyriemann.classification.TSclassifier(metric='riemann')
    TSclassifier.fit(cov_X_train, y_train)
    
    
    y_predict = TSclassifier.predict(cov_X_test)
    print(sklearn.metrics.accuracy_score(TSclassifier.predict(cov_X_train), y_train))
    scores.append(sklearn.metrics.accuracy_score(y_predict, y_test))


class_balance = np.mean(np.asarray(labels) == np.asarray(labels)[0])
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                          class_balance))

(262, 32, 1033)
0.8854961832061069
(262, 32, 1033)
0.8969465648854962
(262, 32, 1033)
0.9007633587786259
(262, 32, 1033)
0.8854961832061069
(262, 32, 1033)
0.8931297709923665
(262, 32, 1033)
0.8931297709923665
(262, 32, 1033)
0.9007633587786259
(262, 32, 1033)
0.9122137404580153
(262, 32, 1033)
0.9045801526717557
(262, 32, 1033)
0.9083969465648855
Classification accuracy: 0.845455 / Chance level: 0.509146


In [17]:
import sklearn
from sklearn.decomposition import PCA
from sklearn import linear_model, decomposition, datasets
from sklearn.metrics import accuracy_score
from sklearn.model_selection import ShuffleSplit  


for n_component in range(30,31):
    sparse_epochs = sparse_epochs.reshape((328, -1))
    pca = PCA(n_components=n_component)
    pca.fit(sparse_epochs)
    total_variance = np.asarray(pca.explained_variance_ratio_)
    print(sum(total_variance))
    new_X = pca.transform(sparse_epochs)

    samples = new_X

    scores = []
    cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
    for train_idx, test_idx in cv.split(samples):
        y_train, y_test = np.asarray(labels)[train_idx], np.asarray(labels)[test_idx]
        X_train = samples[train_idx]
        X_test = samples[test_idx]



#         logistic = linear_model.LogisticRegression(C = 1e-5)
#         logistic.fit(X_train, y_train)
        lda = LinearDiscriminantAnalysis()
        lda.fit(X_train, y_train)



#         y_predict = logistic.predict(X_test)
        y_predict = lda.predict(X_test)
        scores.append(sklearn.metrics.accuracy_score(y_predict, y_test))


    class_balance = np.mean(np.asarray(labels) == np.asarray(labels)[0])
    print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                              class_balance))

0.16448982274364676
Classification accuracy: 0.527273 / Chance level: 0.509146


In [4]:
from sklearn.preprocessing import StandardScaler
import copy


epochs_std = copy.copy(sparse_epochs)


sample_num, chan_num, timepoint = sparse_epochs.shape
for c in range(chan_num):
    original_timepoints = sparse_epochs[:,c,:]
    scaler = StandardScaler()
    scaler.fit(original_timepoints)
    chan_std = scaler.transform(original_timepoints)
    epochs_std[:,c,:] = chan_std
    

print(np.array_equal(epochs_std, sparse_epochs))
print(epochs_std.shape)


False
(328, 19, 3443)


In [5]:
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score

from mne import Epochs, pick_types, find_events
from mne.channels import read_layout
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
from mne.decoding import CSP

#you need to make sure there is no flat channel! otherwise covariance matrix is not positive definite

scores = []
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
for train_idx, test_idx in cv.split(epochs_std):
    y_train, y_test = np.asarray(labels)[train_idx], np.asarray(labels)[test_idx]
    X_train = epochs_std[train_idx]
    X_test = epochs_std[test_idx]
    print(X_train.shape)
    
    csp = CSP(n_components=4, reg=None, log=True, norm_trace=False, cov_est = 'epoch', cov_method_params= 'shrinkage')
    new_epochs = csp.fit_transform(X_train,  y_train)
    print(new_epochs.shape)
    lda = LinearDiscriminantAnalysis()
    lda.fit(new_epochs, y_train)
    
#     logistic = linear_model.LogisticRegression(C = 1e-5)
#     logistic.fit(new_epochs, y_train)
    

    y_pred_csp = csp.transform(X_test)
    y_pred = lda.predict(y_pred_csp)
#     y_predict = logistic.predict(y_pred_csp)
    
    print(y_pred)
    print(y_test)
    print(sklearn.metrics.accuracy_score(y_pred, y_test))
    scores.append(sklearn.metrics.accuracy_score(y_pred, y_test))


class_balance = np.mean(np.asarray(labels) == np.asarray(labels)[0])
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                          class_balance))

(262, 19, 3443)
(262, 4)
[1 0 1 0 1 1 0 0 0 0 0 0 0 1 1 1 0 0 0 1 1 0 0 0 1 1 0 1 0 1 0 1 1 0 1 1 1
 0 0 0 1 0 1 0 1 1 0 0 1 1 1 0 0 0 1 0 1 1 1 1 0 0 0 1 1 1]
[1 0 1 0 1 1 0 0 0 0 0 0 0 1 1 1 0 0 0 1 1 1 0 0 1 1 1 0 0 1 0 1 0 0 1 1 1
 0 0 0 1 0 1 0 1 1 0 1 1 1 1 0 0 0 1 0 1 1 1 0 0 0 0 1 1 1]
0.9090909090909091
(262, 19, 3443)
(262, 4)
[1 1 0 1 1 1 1 0 1 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 1 0 1 0 1 1 0 0 0
 1 0 0 1 1 0 1 1 0 0 1 1 1 0 0 1 0 1 0 0 1 1 0 0 1 0 0 1 0]
[1 0 0 1 1 1 1 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 1 0 1 0 0 0 1 0 0 1 1 1 0 0 0
 1 0 0 1 1 0 1 1 0 0 0 1 1 0 0 1 0 1 0 0 1 1 1 0 1 0 0 1 0]
0.8787878787878788
(262, 19, 3443)
(262, 4)
[1 1 0 1 1 1 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 1 1 1 0 1 0 1 0 1
 0 1 1 1 1 1 0 1 0 0 1 1 1 0 1 1 1 0 0 1 1 0 1 0 1 0 1 0 0]
[1 1 0 1 1 1 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0 1 0 1
 0 0 1 1 1 1 0 1 0 0 1 1 1 0 1 1 1 0 0 1 1 0 1 0 1 1 1 0 0]
0.9393939393939394
(262, 19, 3443)
(262, 4)
[0 1 0 0 1 1 1 0 0 0 0 1 0 1 1 1 

In [31]:
np.unique(labels)

array([0, 1])

In [4]:
a = [[True, True], [True, True]]
np.any(True)

AttributeError: 'list' object has no attribute 'any'

In [13]:
a = np.matmul(sparse_epochs[0,:,:], sparse_epochs[0,:,:].T)

print(np.all(np.linalg.eigvals(a) > 0))


False


In [12]:
a = [[1,2,2,1,2,4,1,4,1,2], [1,2,2,1,22,4,-1,44,41,72]]
a = np.asarray(a)

scaler = StandardScaler()
a_std = scaler.fit_transform(a)
cov = np.matmul(a_std, a.T)
print(np.all(np.linalg.eigvals(cov) > 0))

True


C:\Users\kun\Anaconda2\envs\py36\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int32 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [3]:
flat_chan = [4,7,8,14,15,16,20,21,22,23,24, 31,32] 
flat_chan = [f-1 for f in flat_chan]
good_chan = [i for i in range(0,32) if i not in flat_chan]
sparse_epochs = sparse_epochs[:,good_chan,:]
print(sparse_epochs.shape)

(328, 19, 3443)
